In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from AnsatzLacs import *
from qiskit_algorithms.optimizers import SPSA
from CCAUtil import *

In [ ]:
# implementation of the slack variabel method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 0

problem_seed = 4123
swap_test_unitary = get_swap_unitary(sys_size)

In [ ]:
np.random.seed(problem_seed)
rho_ansatz = General(sys_size, ref_size, 2)
sigma_ansatz = General(sys_size, ref_size, 2)
rho_prob = QCBM(sys_size, ref_size, 1)
sigma_prob = QCBM(sys_size, ref_size, 1)
rho_param = np.random.normal(size=rho_ansatz.get_parameter_num()) % (2 * np.pi)
sigma_param = np.random.normal(size=sigma_ansatz.get_parameter_num()) % (2 * np.pi)
rho_prob_param = np.random.normal(size=rho_prob.get_parameter_num()) % (2 * np.pi)
sigma_prob_param = np.random.normal(size=sigma_prob.get_parameter_num()) % (2 * np.pi)
rho_ansatz.set_params(rho_param)
sigma_ansatz.set_params(sigma_param)
rho_prob.set_params(rho_prob_param)
sigma_prob.set_params(sigma_prob_param)

rho = CCA(rho_prob, rho_ansatz)
sigma = CCA(sigma_prob, sigma_ansatz)

In [ ]:
# calculate the trace distance
def trace_distance(rho, sigma):
    eig, _ = np.linalg.eig(rho - sigma)
    eig = np.abs(eig)
    return 0.5*np.sum(eig)

expected = trace_distance(rho.get_dm(None), sigma.get_dm(None))
print("Exact trace distance:", expected)

In [ ]:
def get_trace(cca1, cca2, qcbm_shots, num_shots):
    # pass the swap_test_unitary to the helper function in CCAUtil
    return get_trace_helper(swap_test_unitary, cca1, cca2, qcbm_shots, num_shots)

In [ ]:
# precompute terms
rho_squared = get_trace(rho, rho, None, None)
sigma_squared = get_trace(sigma, sigma, None, None)
rho_sigma = get_trace(rho, sigma, None, None)

# Dual Optimization

In [ ]:
def dual_cost_function_full(params):
    # update parameters
    lmbda = params[-2]
    mu = params[-1]
    
    omega_params = params[:dual_num_params]
    tau_params = params[dual_num_params:2 * dual_num_params]

    omega.set_params(omega_params)
    tau.set_params(tau_params)
    
    # calculate new objective
    exp_omega = get_purity(omega, qcbm_shots)
    exp_tau =get_purity(tau, qcbm_shots)
    termA = lmbda**2*exp_omega + rho_squared + sigma_squared + mu**2*exp_tau
    
    exp_omega_rho = get_trace(omega, rho, qcbm_shots, num_shots)
    exp_omega_sigma = get_trace(omega, sigma, qcbm_shots, num_shots)
    exp_omega_tau = get_trace(omega, tau, qcbm_shots, num_shots)
    termB = -lmbda*exp_omega_rho + lmbda*exp_omega_sigma - lmbda*mu*exp_omega_tau
    
    exp_rho_tau = get_trace(rho,tau, qcbm_shots, num_shots)
    exp_sigma_tau = get_trace(sigma, tau, qcbm_shots, num_shots)
    termC = -rho_sigma + mu*exp_rho_tau - mu*exp_sigma_tau
    
    obj = lmbda
    penal = termA + 2*(termB + termC)
    
    cost = obj + c * penal
    return cost, obj, penal

def dual_cost_function(params):
    cost, _, _ = dual_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def dual_callback_spsa(nfev, x, fx, stepsize, accepted=False):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global dual_loss
    global dual_penals
    global dual_ojectives
    
    fx, obj, penal = dual_cost_function_full(x)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)
        
    if(nfev % print_every == 0):
        print(f"Iteration: {nfev//3}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {c}")

In [ ]:
num_shots = 1e12 # use None for noiseless simulation
qcbm_shots = 1e12

max_iter = 50 * 1000
c = 100
lmbda = 1; mu = 1
train_lr = 5e-3
pertb = 1e-4
trust = True # Normalize gradient if True

dual_num_layers = 3
dual_qcbm_layers = 2

dual_unitary_params = get_parameter_number("general", sys_size, ref_size, dual_num_layers)
dual_qcbm_params = get_parameter_number("qcbm", sys_size, ref_size, dual_qcbm_layers)
dual_num_params = dual_unitary_params + dual_qcbm_params

In [ ]:
num_runs = 3
np.random.seed(np.random.randint(1e5)) # For random initialization
record = 0 # record the number of successful trails

costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

for i in range(num_runs):
    init_seed = np.random.randint(1e5)
    np.random.seed(init_seed)
    
    dual_loss = []
    dual_penals = []
    dual_objectives = []

    omega = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)
    tau = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)
    
    print(f"Exact value: {expected}")
    
    init_params = [*omega.get_params(), *tau.get_params(), lmbda, mu]
    print(f"Iteration {i}, Initial cost : {dual_cost_function(init_params)}")
    
    autospsa = SPSA(maxiter=max_iter, learning_rate=train_lr,
                        perturbation=pertb, callback=dual_callback_spsa, trust_region=trust)
    obj = autospsa.minimize(fun=dual_cost_function, x0=init_params)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)

    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"TD_CCA_dual_data/seed{problem_seed}"
        problem_name = "TD"
        filename = f"Init_{init_seed}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}")

# Primal Optimization

In [ ]:
def primal_cost_function_full(params):
    # update parameters
    lmbda = params[-2]
    mu = params[-1]
    
    omega_params = params[:primal_num_params]
    tau_params = params[primal_num_params:2 * primal_num_params]

    omega.set_params(omega_params)
    tau.set_params(tau_params)
    
    # calculate new objective
    omega_squared = get_purity(omega, qcbm_shots)
    tau_squared = get_purity(tau, qcbm_shots)
    
    omega_rho = get_trace(rho, omega, qcbm_shots, num_shots)
    omega_sigma = get_trace(sigma, omega, qcbm_shots, num_shots)
    omega_tau = get_trace(omega, tau, qcbm_shots, num_shots)
    
    obj = lmbda * (omega_rho + -omega_sigma)
    
    penaltya = 2 ** sys_size + lmbda ** 2 * omega_squared + mu ** 2 * tau_squared
    penaltyb = lmbda * mu * omega_tau - lmbda - mu
    
    penalty = penaltya + 2 * penaltyb
    
    cost = obj - c * penalty
    return cost, obj, penalty

def primal_cost_function(params):
    cost, _, _ = primal_cost_function_full(params)
    # return -cost for maximization
    return -cost

In [ ]:
print_every = 100
def primal_callback_spsa(nfev, x, fx, stepsize, accepted=False):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global primal_loss
    global primal_penals
    global primal_ojectives
    
    fx, obj, penal = primal_cost_function_full(x)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
        
    if(nfev % print_every == 0):
        print(f"Iteration: {nfev//3}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {c}")

In [ ]:
num_shots = 1e12
qcbm_shots = 1e12

max_iter = 50 * 1000
c = 5
lmbda = 1; mu = 1
lr_max, lr_min = 1e-2, 1e-3
dec_interval = 5000
train_lr = get_lr(lr_max, lr_min, dec_interval, max_iter) # halve the lr every 5000 iterations
pertb = 1e-4
trust = True # Normalize gradient if True

primal_num_layers = 4
primal_qcbm_layers = 2

primal_unitary_params = get_parameter_number("general", sys_size, ref_size, primal_num_layers)
primal_qcbm_params = get_parameter_number("qcbm", sys_size, ref_size, primal_qcbm_layers)
primal_num_params = primal_unitary_params + primal_qcbm_params

In [ ]:
num_runs = 3
np.random.seed(np.random.randint(1e5)) # For random initialization
record = 0 # record the number of successful trails

costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_runs):
    init_seed = np.random.randint(1e5)
    np.random.seed(init_seed)
    
    primal_loss = []
    primal_penals = []
    primal_objectives = []

    omega = create_cca("general", sys_size, ref_size, primal_num_layers, primal_qcbm_layers)
    tau = create_cca("general", sys_size, ref_size, primal_num_layers, primal_qcbm_layers)
    
    print(f"Exact value: {expected}")
    
    init_params = [*omega.get_params(), *tau.get_params(), lmbda, mu]
    print(f"Iteration {i}, Initial cost : {-primal_cost_function(init_params)}")
    
    autospsa = SPSA(maxiter=max_iter, learning_rate=train_lr,
                        perturbation=pertb, callback=primal_callback_spsa, trust_region=trust)
    obj = autospsa.minimize(fun=primal_cost_function, x0=init_params)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)

    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"TD_CCA_primal_data/seed{problem_seed}"
        problem_name = "TD"
        filename = f"Init_{init_seed}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}")

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 500
window_size_dual = 500

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
x1 = len(costs_primal_median)
x2 = len(costs_dual_median)
x3 = max(x1, x2)
expected_line = np.zeros(x3) + expected
plt.plot(np.arange(x1), costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(np.arange(x2), costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(x3), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(np.arange(x1), costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(np.arange(x2), costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(0, .5)
plt.xlim(0, max_iter)

# Display the graph
plt.show()